In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [16]:
# import dataset
import pandas as pd

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# reading each csv file as data frame
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# list 
dfs = [df_21, df_22, df_23]

# concat
df = pd.concat(dfs, ignore_index=True)

# print and chenk result
print(df.head())  # 합쳐진 데이터프레임의 첫 부분 출력
print(df.shape)


       MSR_DT STTN_NM  PM25  PM10     O3   CO    SO2    NO2     NO    NOX
0  2021010101     명서동  11.0  22.0  0.022  0.5  0.003  0.023  0.009  0.032
1  2021010102     명서동  12.0  16.0  0.028  0.4  0.003  0.015  0.007  0.022
2  2021010103     명서동  13.0  21.0  0.031  0.4  0.003  0.013  0.006  0.019
3  2021010104     명서동  15.0  19.0  0.028  0.4  0.003  0.013  0.006  0.019
4  2021010105     명서동  15.0  22.0  0.024  0.4  0.003  0.015  0.006  0.021
(1122344, 10)


In [17]:
import pandas as pd
import numpy as np


# interpolate 
for col in df.columns:
    # if df[col].dtype != 'datetime64[ns]':
    df[col] = df[col].interpolate(method='linear', limit_direction='both')

df_copy = df.copy()

# count nulls
null_counts = df_copy.isnull().sum()

# check null count on each columns
PM25_null_count = df_copy['PM25'].isnull().sum()
PM10_null_count = df_copy['PM10'].isnull().sum()
O3_null_count = df_copy['O3'].isnull().sum()
CO_null_count = df_copy['CO'].isnull().sum()
SO2_null_count = df_copy['SO2'].isnull().sum()
NO2_null_count = df_copy['NO2'].isnull().sum()

print("PM25 컬럼의 NULL 값 개수:", PM25_null_count)
print("PM10 컬럼의 NULL 값 개수:", PM10_null_count)
print("O3 컬럼의 NULL 값 개수:", O3_null_count)
print("CO 컬럼의 NULL 값 개수:", CO_null_count)
print("SO2 컬럼의 NULL 값 개수:", SO2_null_count)
print("NO2 컬럼의 NULL 값 개수:", NO2_null_count)

# print total null count
total_null_count = df_copy.isnull().sum().sum()
print("전체 데이터프레임의 NULL 값 개수:", total_null_count)

/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_71302/960638065.py:8: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[col] = df[col].interpolate(method='linear', limit_direction='both')


PM25 컬럼의 NULL 값 개수: 0
PM10 컬럼의 NULL 값 개수: 0
O3 컬럼의 NULL 값 개수: 0
CO 컬럼의 NULL 값 개수: 0
SO2 컬럼의 NULL 값 개수: 0
NO2 컬럼의 NULL 값 개수: 0
전체 데이터프레임의 NULL 값 개수: 0


In [18]:
df = df[~df['STTN_NM'].isin(['이동차', '부산항'])]
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)
print(df)

             MSR_DT  PM25  PM10     O3   CO    SO2    NO2
0        2021010101  11.0  22.0  0.022  0.5  0.003  0.023
1        2021010102  12.0  16.0  0.028  0.4  0.003  0.015
2        2021010103  13.0  21.0  0.031  0.4  0.003  0.013
3        2021010104  15.0  19.0  0.028  0.4  0.003  0.013
4        2021010105  15.0  22.0  0.024  0.4  0.003  0.015
...             ...   ...   ...    ...  ...    ...    ...
1117136  2023123120   0.0  11.0  0.037  0.4  0.001  0.013
1117137  2023123121  15.0  15.0  0.036  0.4  0.001  0.012
1117138  2023123122   0.0  14.0  0.035  0.4  0.001  0.012
1117139  2023123123   0.0  11.0  0.032  0.4  0.001  0.014
1117140  2023123124   0.0  16.0  0.031  0.4  0.001  0.013

[1096258 rows x 7 columns]


In [19]:
# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 확인
print("Data after scaling:")
print(df.head())

Data after scaling:
       MSR_DT      PM25      PM10        O3        CO       SO2       NO2
0  2021010101  0.041237  0.017460  0.100000  0.097656  0.019737  0.212963
1  2021010102  0.044674  0.012698  0.127273  0.078125  0.019737  0.138889
2  2021010103  0.048110  0.016667  0.140909  0.078125  0.019737  0.120370
3  2021010104  0.054983  0.015079  0.127273  0.078125  0.019737  0.120370
4  2021010105  0.054983  0.017460  0.109091  0.078125  0.019737  0.138889


In [20]:
# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 80:20으로 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (877006, 4)
Shape of X_test: (219252, 4)
Shape of y_train: (877006, 2)
Shape of y_test: (219252, 2)


In [21]:
# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())


/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. 

         test_PM25  pred_PM25  test_PM10  pred_PM10
118080    0.065292        NaN   0.016667        NaN
320506    0.030928        NaN   0.009524        NaN
1001690   0.171821        NaN   0.100794        NaN
40205     0.034418        NaN   0.012717        NaN
124805    0.044674        NaN   0.042063        NaN


문제 설명 및 해석
경고 메시지:

ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.:
SARIMAX 모델이 예측할 때 지원되지 않는 인덱스가 제공되었음을 의미합니다. 예측 결과에 인덱스를 무시하고 있습니다.
ConvergenceWarning: Maximum Likelihood optimization failed to converge.:
최대 우도 최적화가 수렴하지 못했음을 의미합니다. 모델이 최적의 파라미터를 찾지 못했을 수 있습니다.
예측 결과:

예측 값(pred_PM25, pred_PM10)이 NaN으로 나왔습니다. 이는 모델이 예측을 생성하지 못했다는 것을 의미합니다.
원인 분석
지원되지 않는 인덱스:

X_train 및 X_test 데이터에 시간 인덱스가 포함되어 있지 않거나, 모델이 이를 지원하지 않는 형식으로 인덱스를 제공받았을 가능성이 큽니다.
최적화 실패:

데이터가 정상적으로 제공되지 않거나, 모델의 설정이 잘못되었을 수 있습니다. 특히, SARIMAX 모델은 시계열 데이터에서 특정 파라미터 설정에 민감할 수 있습니다.
해결 방법
인덱스 문제 해결:

X_train 및 X_test의 인덱스를 올바르게 설정합니다. 시계열 데이터에서 적절한 인덱스를 설정하여 모델이 이를 인식할 수 있도록 합니다.
모델 설정 및 데이터 검토:

모델의 order 파라미터를 조정하거나 데이터를 다시 검토합니다. 특정 파라미터가 모델의 수렴에 영향을 미칠 수 있습니다.
데이터의 스케일링, 타입 변환 등을 다시 한 번 확인하여 올바르게 제공되었는지 검토합니다.


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error



# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# reading each csv file as data frame
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# list 
dfs = [df_21, df_22, df_23]

# concat
df = pd.concat(dfs, ignore_index=True)

# 불필요한 열 제거
df = df[~df['STTN_NM'].isin(['이동차', '부산항'])]
df.drop(columns=['NO', 'NOX', 'STTN_NM'], inplace=True)

# MinMaxScaler 사용
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# 데이터 타입 확인 및 변환
df[numeric_columns] = df[numeric_columns].astype(float)
# df['Year'] = df['Year'].astype(int)
# df['Month'] = df['Month'].astype(int)
# df['Hour'] = df['Hour'].astype(int)

# 타겟 변수 설정 (예시로 PM25와 PM10 예측)
target_columns = ['PM25', 'PM10']
X = df.drop(columns=target_columns + ['MSR_DT'])  # 예측에 사용될 피처
y = df[target_columns]  # 타겟 변수

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 인덱스 확인 및 설정
X_train.index = pd.to_datetime(df['MSR_DT'][X_train.index])
X_test.index = pd.to_datetime(df['MSR_DT'][X_test.index])
y_train.index = pd.to_datetime(df['MSR_DT'][y_train.index])
y_test.index = pd.to_datetime(df['MSR_DT'][y_test.index])

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# SARIMAX 모델 설정 및 학습
# PM25 예측 모델
model_pm25 = SARIMAX(y_train['PM25'], exog=X_train, order=(1, 1, 1))
results_pm25 = model_pm25.fit(disp=False)

# PM10 예측 모델ß
model_pm10 = SARIMAX(y_train['PM10'], exog=X_train, order=(1, 1, 1))
results_pm10 = model_pm10.fit(disp=False)

# 예측
pred_pm25 = results_pm25.get_prediction(exog=X_test).predicted_mean
pred_pm10 = results_pm10.get_prediction(exog=X_test).predicted_mean

# 예측 결과를 데이터프레임으로 저장
test_pred = pd.DataFrame({'test_PM25': y_test['PM25'], 'pred_PM25': pred_pm25,
                          'test_PM10': y_test['PM10'], 'pred_PM10': pred_pm10}, index=y_test.index)

print(test_pred.head())

# 성능 평가
mse_pm25 = mean_squared_error(y_test['PM25'], pred_pm25)
mse_pm10 = mean_squared_error(y_test['PM10'], pred_pm10)

print(f"Mean Squared Error for PM25: {mse_pm25}")
print(f"Mean Squared Error for PM10: {mse_pm10}")


Shape of X_train: (877006, 4)
Shape of X_test: (219252, 4)
Shape of y_train: (877006, 2)
Shape of y_test: (219252, 2)


MissingDataError: exog contains inf or nans